In [2]:
import nltk
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
nltk.download('brown')
nltk.download('universal_tagset')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /home/parth/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/parth/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [8]:
START_DELIM = '^'

In [158]:
tagged_sentences = brown.tagged_sents(tagset='universal')
all_tags = list(set([tag for sentence in tagged_sentences for _, tag in sentence]))

In [161]:
class HiddenMarkovModel:

    def __init__(self, sentences = []):
        self.sentences = sentences
        self._find_tags()
        self.compute_transition_probabilities()
        self.compute_emmision_probabilities()
    
    def _find_tags(self):
        self.tags = list(set( [ word_tag[1] for sentence in tagged_sentences for word_tag in sentence ] ))
    
    def compute_transition_probabilities(self):
        T = defaultdict(lambda: defaultdict(lambda:0))
        # T = dict of dicts
        # T[tcurr] = dict
        # T[tcurr][tprev] = integet with default value as 0
        TC = defaultdict(lambda : 0)
        # TC (tag count) = dict
        # TC[tag] = integer with default value as 0
        
        for sentence in self.sentences:
            # skip first word for now
            prev_word = (START_DELIM, START_DELIM)
            for word in sentence:
                T[word[1]][prev_word[1]] += 1
                TC[word[1]] += 1
                prev_word = word
        
        TC[START_DELIM] = len(self.sentences)

        for curr_tag in T:
            for prev_tag in T[curr_tag]:
                if curr_tag in T and prev_tag in T[curr_tag]:
                    T[curr_tag][prev_tag] = T[curr_tag][prev_tag] / TC[prev_tag]
        
        self.T = T
        self.TC = TC
    
    def compute_emmision_probabilities(self):
        W = defaultdict(lambda: defaultdict(lambda:0))
        # W = dict of dicts
        # W[word] = dict
        # W[word][tag] = integet with default value as 0        

        for sentence in self.sentences:
            for word in sentence:
                W[word[0]][word[1]] += 1

        for word in W.keys():
            for tag in W[word].keys():
                W[word][tag] = W[word][tag] / self.TC[tag]

        self.W = W


In [162]:
def Viterbi(sentence, hmm):
    selected_path_and_probs = [{ '^' : ('^' , 1) }]
    # tag : ( best_prev_tag , best_prob_if_word_has_this_tag )
    
    
    curr_state_probabilities = {}
    for tag in all_tags:
        # any tag can be placed for the first word w1
        curr_state_probabilities[tag] = ('^', hmm.T[tag].get('^',1e-6) * hmm.W[sentence[0]].get(tag, 1e-6))
        
    selected_path_and_probs.append(curr_state_probabilities)
    
    for word in sentence[1:]:
        curr_state_probabilities = {}
        for curr_tag in all_tags:
            curr_max = None
            for prev_tag , best_till_prev in selected_path_and_probs[-1].items():
                possible_state_prob = best_till_prev[1] * hmm.T[curr_tag].get(prev_tag,1e-6) * hmm.W[word].get(curr_tag, 1e-6)
                if not curr_max or possible_state_prob > curr_max:
                    curr_state_probabilities[curr_tag] = (prev_tag , possible_state_prob)
                    curr_max = possible_state_prob
                    
        selected_path_and_probs.append(curr_state_probabilities)
    
    
    max_prob = max([ d[1] for d in selected_path_and_probs[-1].values()])
    last_tag = None
    
    for tag in selected_path_and_probs[-1].keys():
        if selected_path_and_probs[-1][tag][1] == max_prob:
            last_tag = tag
    
    rev_tag_seq = [last_tag]

    for i in range(len(sentence)-1):
        ind = -1*(i + 1)
        last_tag = selected_path_and_probs[ind][last_tag][0]
        rev_tag_seq.append(last_tag)

    
    return [w for w in reversed(rev_tag_seq)]



In [168]:
# perform k-fold cross validation
NUM_FOLDS = 5
kf = KFold(NUM_FOLDS , shuffle=  True, random_state=42)

accuracies = []
all_true_tags = []
all_predicted_tags = []


for train_indices, test_indices in kf.split(tagged_sentences):
    train_data = [tagged_sentences[i] for i in train_indices]
    test_data = [tagged_sentences[i] for i in test_indices]

    hmm = HiddenMarkovModel(train_data)

    true_tags = []
    predicted_tags = []
    
    for sentence in test_data:
        words = [word[0] for word in sentence]
        true_tags += [word[1] for word in sentence]
        predicted_tags += Viterbi(words,hmm)
    
    all_true_tags += true_tags
    all_predicted_tags += predicted_tags
    
    accuracies.append(accuracy_score(true_tags, predicted_tags))

print(accuracies)
overall_accuracy = np.mean(accuracies)
print(f"Overall Accuracy (5-Fold Cross-Validation): {overall_accuracy:.4f}")

report = classification_report(all_true_tags, all_predicted_tags, labels=all_tags)
print("Per POS Tag Accuracy:\n", report)


[0.9558093710520983, 0.9564629191601932, 0.9563745573224082, 0.9561343535119081, 0.9560339919448368]
Overall Accuracy (5-Fold Cross-Validation): 0.9562
Per POS Tag Accuracy:
               precision    recall  f1-score   support

         NUM       0.97      0.91      0.94     14874
        NOUN       0.96      0.95      0.95    275558
         ADJ       0.91      0.91      0.91     83721
        VERB       0.97      0.94      0.96    182750
         DET       0.95      0.99      0.97    137019
           X       0.80      0.49      0.60      1386
         ADV       0.91      0.89      0.90     56239
         PRT       0.91      0.91      0.91     29829
         ADP       0.95      0.97      0.96    144766
        PRON       0.95      0.98      0.97     49334
        CONJ       0.99      0.99      0.99     38151
           .       0.99      1.00      1.00    147565

    accuracy                           0.96   1161192
   macro avg       0.94      0.91      0.92   1161192
weighted avg 

In [99]:
tags_seq = Viterbi("The boat was sailing in the pacific ocean".split(" "),hmm)
print(' '.join(tags_seq))

^ DET NOUN VERB VERB ADP DET ADJ NOUN
